In [1]:
%cd /ibex/user/slimhy/PADS/code
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import torch

import util.misc as misc
import models.s2vs as ae_mods


def get_args_parser():
    parser = argparse.ArgumentParser("Extracting Features", add_help=False)

    # Model parameters
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU"
        " (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits"],
        help="dataset name",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient "
        "(sometimes) transfer to GPU.",
    )

    return parser


# Set dummy arg string to debug the parser
call_string = """--ae_pth ckpt/ae_m512.pth \
    --ae kl_d512_m512_l8 \
    --ae-latent-dim 4096 \
    --batch_size 32 \
    --num_workers 8 \
    --device cuda"""
    

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())

# --------------------
device = torch.device(args.device)

# Fix the seed for reproducibility
misc.set_all_seeds(args.seed)

torch.backends.cudnn.benchmark = True
# --------------------

# Instantiate autoencoder
ae = ae_mods.__dict__[args.ae]()
ae.eval()
print("Loading autoencoder %s" % args.ae_pth)
ae.load_state_dict(torch.load(args.ae_pth, map_location="cpu")["model"])
ae = ae.to(device)

# Compile using torch.compile
ae = torch.compile(ae, mode="max-autotune")

/ibex/user/slimhy/PADS/code
Set seed to 0
Loading autoencoder ckpt/ae_m512.pth


In [2]:
from datasets.latents import ShapeLatentDataset, ComposedPairedShapesLoader

class PairType():
    ROT_PAIR = "rand_no_rot,rand_no_rot"
    PART_DROP = "part_drop,orig"

latents_dir = "/ibex/project/c2273/PADS/3DCoMPaT"

# Create your dataset
dataset = ShapeLatentDataset(latents_dir, split="train", shuffle_parts=False)

# Create the DataLoader using the sampler
dataloader = ComposedPairedShapesLoader(
    dataset,
    batch_size=32,
    pair_types_list=[PairType.ROT_PAIR, PairType.PART_DROP],
    num_workers=0,
    shuffle=False,
) 

# Use the dataloader in your training loop
k_break = 1
k = 0
for pair_types, (l_A, bb_A, bb_l_A, meta_A), (l_B, bb_B, bb_l_B, meta_B) in dataloader:
    k += 1
    if k == k_break:
        break

In [3]:
from models.partvae import PartAwareVAE

# Compute the mask from batch labels
mask_A = bb_l_A != -1
mask_A = mask_A.to(device) # B x 24

l_A = l_A.to(device)       # B x 512 x 8
bb_A = bb_A.to(device)     # B x 24 x 4 x 3
bb_l_A = bb_l_A.to(device) # B x 24

dummy = PartAwareVAE(
    dim=512,
    latent_dim=128,
    heads=8,
    dim_head=64,
    depth=2,
).to(device)
dummy = dummy.to(device)

# Forward pass
logits, kl = dummy(latents=l_A, part_bbs=bb_A, part_labels=bb_l_A, batch_mask=mask_A)

torch.Size([16, 24, 1024])


In [4]:
# Print param count in human readable format
print("Model param count: ", misc.count_params(dummy))

# Print param count in human readable format
print("Model param count: ", misc.count_params(ae))

Model param count:  20,171,080
